# Database record design in yabadaba

While yabadaba does its best to abstract the data from the database infrastructure, there are still some infrastructure limitations that need to be considered when designing records.  


The main consideration is whether data should be directly stored in the record itself, or stored separately in the database and connected with the record.

- The data records themselves are best at storing simple types of data like single numbers, strings and boolean values.  Some examples of this are input settings, metadata descriptor fields, and single-valued outputs.  Directly including these fields in the records makes it possible to query the database to return only the records that have certain values or ranges of values of these terms.
- It is best to store large, complex data separately in the database in their original format with the record serving as the reference point for accessing and finding the content.  Some examples are images, large tabulated data files, binary files, or any other complex data that is not easy to convert.  The downside is that the database cannot perform query operations on the raw data because the database does not know how to interpret the data.
- The two categories above are at extreme ends of what data can look like, and there is some intermediate gray area for moderately complex data.  For instance, the x and y coordinates of a linear plot can easily be included in JSON/XML as arrays.  While it is impractical to query the database on such data, it can still be useful to include the data directly in the record as it will instantly be accessible to anyone who downloads the record without needing to then download a separate file from the database.  The opposing argument is that having large amounts of data directly in the records adversely affects performance of both query operations and RAM usage, which limits the number of records that can be downloaded and interacted with at any time.  Deciding what to do with this type of data depends on how large the moderately complex data is, how many records you will likely have that contains such data, and if instant access to the content is beneficial.

## Parameter objects for simple in-record data

There are a number of Parameter objects in yabadaba to help with the handling of the simple types of data that are in-record values.  The Parameter objects make it so you define the type of data field, what you want to call it, and then most of everything else is automatically set up for converting the data between different formats.

How to define a data field with a Parameter:

1. 'from yabadaba.parameters import *' or the specific Parameter subclasses you plan on using.
2. In your Record object's \_\_init\_\_(), define Record attributes that are Parameter objects.  When you initialize a Parameter, the class specifies the data type, then you give it a name, the record self, and optionally a default value.
3. Define a tuple named parameters that includes all of the Parameter objects.  Note that the order of the parameters is important as they are iterated over when building the record contents: the fields for the earlier parameters will show up above the later parameters!
4. Call super().\_\_init\_\_() passing through the arguments of the current Record class's init and the parameters tuple.
5. Later in your code, define class properties and setters for each of the Parameters where they get/set the value attribute of the corresponding Parameter objects.

(Plans: Steps 2-4 are to be streamlined, possibly into a "parameters" dict or init_parameters() method) 

These simple Parameter types are all associated with a specific default Query operation.  The default Query can be replaced and/or additional queries associated with the Parameter can be added.  See Query documentation below.


## Parameter setup of moderately complex in-record data

Some Parameter objects also exist for some of the more basic moderately complex data types, such as lists and numpy arrays.  Depending on how complex your data is, you may need to define your own Parameter subclass for the data to define the associated data transformation operations (and optionally Query operations).

## Interacting with complex data

The key aspect of interacting with complex data associated with a record is that there needs to be some scheme for identifying where the content is stored in the database from knowing the record.  There are two general ways that you can do this

### Access data in the record's tar file

Each record is designed to primarily access and interact with a single tar file that is named for the record.  Any files contained within the tar file can be retrieved and read in, or the entire contents of the tar file can be extracted. Operations that support this

- Database classes have get_tar, add_tar, update_tar, and delete_tar methods for accessing, adding, modifying, and removing a record's tar file.
- Record classes have a database attribute that ties that record to a specific database.  If you retrieve the record from a database, this is automatically set to the database that was used.
- Building on this, Record classes also have a tar attribute that is a TarFile object of the record's tar.  The first time tar is called for the record, the corresponding tar contents are retrieved from the database.  The stored value can then be cleared with clear_tar.
- Records also have a get_file() method that will retrieve a file from the tar based on the file name.

Comments: This design is best suited for collecting and archiving all files in a folder when a record is completed rather than incrementally adding new files to existing records.  Files can be added to the tars, but it requires downloading/extracting the current tar, adding the new files, then uploading and replacing the tar contents.  (It might be possible to add a simple "add_file" method to a record to help automate this.)

### Access data stored as independent files

Alternatively, files can be independently added to the database.  The challenge here is that there is no default scheme for associating the files with a particular record.  You need some way of uniquely naming/identifying the files, and connecting those files to a record in such a way that a Record object can find them.  Following the tar scheme above, you could name the files based on the record's unique name and some additional attribute.  If the additional attribute is a constant across all records, then that is enough for a specific record to find the content.  Alternatively, you would need to add fields to your record that indicate where the files are located in one or more particular databases.

(File parameters to be added...)






